In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import process
from tqdm import tqdm

/Users/captn_jp/miniconda3/envs/crash_analyzer/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [29]:
# Import dataset
df = pd.read_csv('data/crash_reporting_drivers_data.csv')
df.head()

/var/folders/1b/vcjnk6kd037gn7jddj1yk3_c0000gn/T/ipykernel_96073/879525709.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/crash_reporting_drivers_data.csv')


,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,...,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location
0,MCP3040003N,190026050,Montgomery County Police,Property Damage Crash,05/31/2019 03:00:00 PM,NaN,NaN,NaN,NaN,PARKING LOT OF 3215 SPARTAN RD,...,15,No,No,2004,HONDA,TK,UNKNOWN,39.150044,-77.063089,"(39.15004368, -77.06308884)"
1,MCP1307000K,190024786,Montgomery County Police,Property Damage Crash,05/24/2019 05:00:00 PM,NaN,NaN,NaN,NaN,PARKING LOT,...,0,No,No,0,UNK,UNK,NaN,39.199047,-77.250743,"(39.19904667, -77.25074333)"
2,MCP2846008X,230034260,Montgomery County Police,Property Damage Crash,07/17/2023 10:45:00 AM,County,SELFRIDGE RD,County,RANDOLPH RD,NaN,...,35,No,No,2003,FORD,TK,NO MISUSE,39.054588,-77.085974,"(39.05458848, -77.08597423)"
3,MCP32610017,230034668,Montgomery County Police,Property Damage Crash,07/20/2023 11:40:00 PM,Maryland (State),MUNCASTER MILL RD,County,SHADY GROVE RD,NaN,...,45,No,No,2023,TOYT,CP,NO MISUSE,39.148721,-77.147111,"(39.14872076, -77.14711061)"
4,EJ78520081,230033429,Gaithersburg Police Depar,Property Damage Crash,07/13/2023 05:40:00 PM,Municipality,PERRY PKWY,Unknown,ENT TO SHOPPING CENTER,NaN,...,25,No,No,2001,KENWORTH,TRUCK,NaN,39.149085,-77.210731,"(39.14908542, -77.21073135)"


In [3]:
# Print all the 43 column names
df.columns

Index(['Report Number', 'Local Case Number', 'Agency Name', 'ACRS Report Type',
       'Crash Date/Time', 'Route Type', 'Road Name', 'Cross-Street Type',
       'Cross-Street Name', 'Off-Road Description', 'Municipality',
       'Related Non-Motorist', 'Collision Type', 'Weather',
       'Surface Condition', 'Light', 'Traffic Control',
       'Driver Substance Abuse', 'Non-Motorist Substance Abuse', 'Person ID',
       'Driver At Fault', 'Injury Severity', 'Circumstance',
       'Driver Distracted By', 'Drivers License State', 'Vehicle ID',
       'Vehicle Damage Extent', 'Vehicle First Impact Location',
       'Vehicle Second Impact Location', 'Vehicle Body Type',
       'Vehicle Movement', 'Vehicle Continuing Dir', 'Vehicle Going Dir',
       'Speed Limit', 'Driverless Vehicle', 'Parked Vehicle', 'Vehicle Year',
       'Vehicle Make', 'Vehicle Model', 'Equipment Problems', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')

In [4]:
# Function to print the unique values and their frequency in each column in tablular format also prints the number of NA values in each column
def print_unique_values(df):
    for col in df.columns:
        info_series = df[col].value_counts()
        print(info_series)
        print('NA values: ', df[col].isna().sum())
        print('Max freq: ', max(info_series))
        print('Min freq: ', min(info_series))
        print('len: ', len(info_series))
        print('\n__________________________\n')
print_unique_values(df)

Report Number
MCP229800RC    10
MCP2693003Q     9
MCP12130045     9
MCP28810040     8
MCP2667000H     8
               ..
MCP12130055     1
MCP33200017     1
MCP29620050     1
MCP2361001B     1
MCP3090006L     1
Name: count, Length: 94388, dtype: int64
NA values:  0
Max freq:  10
Min freq:  1
len:  94388

__________________________

Local Case Number
200022715    9
180051879    9
190009085    7
220055904    7
16046398     7
            ..
16027040     1
16028341     1
15000578     1
16021525     1
220047494    1
Name: count, Length: 115683, dtype: int64
NA values:  0
Max freq:  9
Min freq:  1
len:  115683

__________________________

Agency Name
Montgomery County Police     134943
MONTGOMERY                    10659
Rockville Police Departme      9514
Gaithersburg Police Depar      7430
Takoma Park Police Depart      2877
Maryland-National Capital      1122
ROCKVILLE                       630
GAITHERSBURG                    390
TAKOMA                          157
MCPARK                

## Pre-Processing

In [60]:
df = pd.read_csv('data/crash_reporting_drivers_data.csv')

df['Crash Date/Time'] = pd.to_datetime(df['Crash Date/Time'], format='%m/%d/%Y %H:%M:%S %p')

# remove_leading_trailing_whitespaces
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip()

/var/folders/1b/vcjnk6kd037gn7jddj1yk3_c0000gn/T/ipykernel_96073/434018207.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/crash_reporting_drivers_data.csv')


In [100]:
# VEHICLE YEAR
# Vehicle Year: sanitize the data and convert to datetime
df = pd.read_csv('data/crash_reporting_drivers_data.csv')

def display_unique_values(df,col, max=True):
    if col not in df.columns:
        print(f"{col} not in dataframe")
        return
    if not max:
        print(df[col].value_counts())
    else:
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(df['Vehicle Year'].value_counts())

    print(f"\nLength of Unique Values: {len(df[col].value_counts())}")
    print(f"\nColumn Length: {len(df[col])}")

display_unique_values(df, 'Vehicle Year')
df['Vehicle Year'] = df['Vehicle Year'].astype(str)

def update_vehicle_year(value):
    if len(value) == 1:
        return '200' + value
    elif len(value) == 2 and value.startswith('20'):
        return '20' + value
    elif len(value) == 2 and value.startswith('99'):
        return '1999'
    elif len(value) == 3 and value.startswith('2'):
        return '20' + value[-2]
    elif len(value) == 3 and value.startswith('9'):
        return '1' + value
    else:
        return value

df['Vehicle Year'] = df['Vehicle Year'].apply(update_vehicle_year)
df['Vehicle Year'] = df['Vehicle Year'].astype(int)
for index, row in df.iterrows():
    if row['Vehicle Year'] > 2023 or row['Vehicle Year'] <= 1900:
        df.at[index, 'Vehicle Year'] = 1800

print("\n__________________________\n")
df['Vehicle Year'] = df['Vehicle Year'].astype(str)
df['Vehicle Year'] = pd.to_datetime(df['Vehicle Year'], format='%Y')
display_unique_values(df, 'Vehicle Year')

/var/folders/1b/vcjnk6kd037gn7jddj1yk3_c0000gn/T/ipykernel_96073/2619957723.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/crash_reporting_drivers_data.csv')


Vehicle Year
2015    11573
2014    11406
2016    10901
2013    10704
2012     9154
2017     8788
2011     8229
2007     7808
2008     7759
2006     7309
2010     7290
2009     6661
2018     6473
2005     6419
2004     6079
2003     5294
2019     5188
2002     4225
0        3875
2001     3449
2020     3255
2000     3227
2021     2369
1999     2164
1998     1619
2022     1448
1997     1271
1996      800
1995      617
2023      507
1994      436
1993      300
1992      196
1991      154
1990      147
1989      110
1988       90
1987       64
1900       63
9999       59
1986       38
1985       33
99         26
2024       25
1983       15
1984       12
1978       10
1982        9
1980        9
1969        5
1974        5
1972        4
1111        4
1968        3
15          3
1979        3
2996        3
2914        3
1901        3
1981        3
1975        3
1966        3
2033        3
1970        3
1955        2
1977        2
1014        2
2912        2
1976        2
2104        2
1234   

In [51]:
# Vehicle Make: Replace values with the most similar company name

# display the whole series: df['Vehicle Make'].value_counts()
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(df['Vehicle Make'].value_counts())
print(len(df['Vehicle Make'].value_counts()))

# Actual company names
companies = ['FORD', 'LEXUS', 'TOYOTA', 'HYUNDAI', 'CHEVROLET', 'HONDA', 'NISSAN', 'DODGE', 'JEEP', 'KIA', 'BMW', 'MAZDA', 'SUBARU', 'GMC', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'ACURA', 'AUDI', 'CADILLAC', 'INFINITI', 'MITSUBISHI', 'CHRYSLER', 'BUICK', 'VOLVO', 'PONTIAC', 'LINCOLN', 'SATURN', 'MINI', 'SCION', 'PORSCHE', 'FIAT', 'MERCURY', 'SUZUKI', 'SMART', 'JAGUAR', 'LAND ROVER', 'SAAB', 'HUMMER', 'FERRARI', 'BENTLEY', 'ASTON MARTIN', 'ROLLS-ROYCE', 'LAMBORGHINI', 'MASERATI', 'ALFA ROMEO', 'DAEWOO', 'ISUZU', 'PORSC', 'HINO', 'FREIGHTLINER', 'KENWORTH', 'UD', 'MACK', 'STERLING', 'BLUE BIRD', 'THOMAS', 'INTERNATIONAL', 'WHITE', 'AMERICAN MOTORS', 'DAIHATSU', 'EAGLE', 'GEO', 'LOTUS', 'MAYBACH', 'MCLAREN', 'YAMAHA', 'TRIUMPH']

# Improvement: 1844 -> 931 unique values
replacement_dict = {}
replaced_counts = 0
for value in df['Vehicle Make'].unique():
    try:
        match = process.extractOne(value, companies)
        if match[1] >= 70:
            replacement_dict[value] = match[0]
            # print(f"{value} => {match[0]}")
            replaced_counts += 1
    except Exception as e:
        # print(f"Exception: {value}")
        continue
print(f"Replaced {replaced_counts} values")

df['Vehicle Make'] = df['Vehicle Make'].replace(replacement_dict)
# print("\n__________________________\n")
print(len(df['Vehicle Make'].value_counts()))


# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(df['Vehicle Make'].value_counts())


1844
HONDA => HONDA
FORD => FORD
TOYOTA => TOYOTA
KENWORTH => KENWORTH
GMC => GMC
KIA => KIA
SUBA => SUBARU
BMW => BMW
HYUNDAI => HYUNDAI
VOLK => VOLKSWAGEN
VOLKSWAGON => VOLKSWAGEN
DODGE => DODGE
NISSAN => NISSAN
LEXUS => LEXUS
YAMAHA => YAMAHA
INFI => INFINITI
MAZDA => MAZDA
TOYTA => TOYOTA
SUBARU => SUBARU
ACUR => ACURA
HYUN => HYUNDAI
FREIGHT => FREIGHTLINER
CHEVROLET => CHEVROLET
INFINITY => INFINITI
VOLVO => VOLVO
CHRYSLERER => CHRYSLER
HOND => HONDA
ACURA => ACURA
DODG => DODGE
MERZ => HUMMER
CHEVY => CHEVROLET
INFINITI => INFINITI
JAGUAR => JAGUAR
MERCEDEZ => MERCEDES-BENZ
JEEP => JEEP
CHEV => CHEVROLET
MAZD => MAZDA
FREIGHTLINER => FREIGHTLINER
HINO => HINO
RAM => ACURA
NISS => NISSAN
VOLKSWAGEN => VOLKSWAGEN
AL => ALFA ROMEO
PORS => PORSCHE
SATURN => SATURN
TOYTOA => TOYOTA
MERCURY => MERCURY
INT => INTERNATIONAL
CADILLAC => CADILLAC
CHRYSLER => CHRYSLER
HYUNDIA => HYUNDAI
VOLKS => VOLKSWAGEN
THOMAS => THOMAS
MOZDA => MAZDA
MERCEDES => MERCEDES-BENZ
THMS => THOMAS
MACK => MAC

FRIEIGHTLINER => FREIGHTLINER
VOLKSAWAGON => VOLKSWAGEN
ROEN => MCLAREN
FRONTLINER => FREIGHTLINER
TL => BENTLEY
LNDROVER => LAND ROVER
KAWAZAKI => KIA
HYUUNDAI => HYUNDAI
SUZUK => SUZUKI
TOYOTAQ => TOYOTA
MITI => MINI
TOYOVAL2002 => TOYOTA
CADILLIC => CADILLAC
ACURA MDX => ACURA
VOLVVAL2008 => VOLVO
KENWTH => KENWORTH
SUBUR => SUBARU
RECON ISUZU => ISUZU
MARIN => ASTON MARTIN
FRD => FORD
STERL => STERLING
TOYIOTA => TOYOTA
SURARU => SUBARU
MERCEDES-BE => MERCEDES-BENZ
MERCEDES ENZ => MERCEDES-BENZ
HT => FREIGHTLINER
PONITAC => PONTIAC
JEEPVAL2013 => JEEP
FREIGHT LINNER => FREIGHTLINER
RANG ROVER => LAND ROVER
MERCEDES BEN => MERCEDES-BENZ
BM => BMW
MECEDEZ => MERCEDES-BENZ
MITSUBICHI => MITSUBISHI
INFIINITY => INFINITI
FO => FORD
FREIGHTL => FREIGHTLINER
COLVO => VOLVO
CHERVEOLET => CHEVROLET
VOLKSWAGKON => VOLKSWAGEN
SUBRA => SUBARU
VOLVO SAB => VOLVO
SUZKI => SUZUKI
RANGER ROVER => LAND ROVER
F0RD => FORD
RDX => BLUE BIRD
KISA => KIA
ROUGE => GEO
TOYOTA4 => TOYOTA
FREITLINER => FREI

In [55]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df['Vehicle Year'].value_counts())

Vehicle Year
1970-01-01 00:00:00.000002015    11573
1970-01-01 00:00:00.000002014    11406
1970-01-01 00:00:00.000002016    10901
1970-01-01 00:00:00.000002013    10704
1970-01-01 00:00:00.000002012     9154
1970-01-01 00:00:00.000002017     8788
1970-01-01 00:00:00.000002011     8229
1970-01-01 00:00:00.000002007     7808
1970-01-01 00:00:00.000002008     7759
1970-01-01 00:00:00.000002006     7309
1970-01-01 00:00:00.000002010     7290
1970-01-01 00:00:00.000002009     6661
1970-01-01 00:00:00.000002018     6473
1970-01-01 00:00:00.000002005     6419
1970-01-01 00:00:00.000002004     6079
1970-01-01 00:00:00.000002003     5294
1970-01-01 00:00:00.000002019     5188
1970-01-01 00:00:00.000002002     4225
1970-01-01 00:00:00.000000000     3875
1970-01-01 00:00:00.000002001     3449
1970-01-01 00:00:00.000002020     3255
1970-01-01 00:00:00.000002000     3227
1970-01-01 00:00:00.000002021     2369
1970-01-01 00:00:00.000001999     2164
1970-01-01 00:00:00.000001998     1619
1970-01-01 0

In [54]:
# unknown_values = ['UNK', 'UNK.', 'UNKNO', 'UNKNOWN', 'UNKNOW', 'UNKN', 'UNKNOW', 'UNKNW', 'UNKNWN', 'UNKNWON', 'UNKNWOWN', 'UNKNWO, UNKNWOWN', 'UNKNWOWN', 'UNKONW, UNKNWOWN', 'UNKNWON']

Report Number                     0
Local Case Number                 0
Agency Name                       0
ACRS Report Type                  0
Crash Date/Time                   0
Route Type                        0
Road Name                         0
Cross-Street Type                 0
Cross-Street Name                 0
Off-Road Description              0
Municipality                      0
Related Non-Motorist              0
Collision Type                    0
Weather                           0
Surface Condition                 0
Light                             0
Traffic Control                   0
Driver Substance Abuse            0
Non-Motorist Substance Abuse      0
Person ID                         0
Driver At Fault                   0
Injury Severity                   0
Circumstance                      0
Driver Distracted By              0
Drivers License State             0
Vehicle ID                        0
Vehicle Damage Extent             0
Vehicle First Impact Locatio